In [1]:
import pandas as pd
import numpy as np
import soccerdata as sd 
import requests
import json
import itertools
import mezzala
import fotmob_scraper as fms
import pandas_montecarlo

[07/15/22 09:50:41] INFO     No custom team name replacements found. You can configure these in       ]8;id=666116;file:///usr/local/lib/python3.9/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=297101;file:///usr/local/lib/python3.9/site-packages/soccerdata/_config.py#85\85]8;;\
                             /Users/keltim01/soccerdata/config/teamname_replacements.json.                         

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=73149;file:///usr/local/lib/python3.9/site-packages/soccerdata/_config.py\_config.py]8;;\:]8;id=285331;file:///usr/local/lib/python3.9/site-packages/soccerdata/_config.py#155\155]8;;\
                             /Users/keltim01/soccerdata/config/league_dict.json.                                   

In [2]:
mh = pd.read_csv('data/super_league_matches_historical.csv')

In [3]:
mh['Home'].unique()

array(['Servette', 'Thun', 'Grasshoppers', 'Luzern', 'St. Gallen',
       'Basel', 'Lausanne', 'Sion', 'Young Boys', 'Zurich', 'Aarau',
       'Vaduz', 'Lugano', 'Xamax'], dtype=object)

In [4]:
url = 'https://raw.githubusercontent.com/openfootball/football.json/master/2020-21/ch.1.json'


In [5]:
from urllib import response


response = requests.get(url).json()

data = response['matches']
data[0:3]

[{'round': 'Spieltag 1',
  'date': '2020-09-19',
  'team1': 'FC Lugano',
  'team2': 'FC Luzern',
  'score': {'ft': [2, 1]}},
 {'round': 'Spieltag 1',
  'date': '2020-09-19',
  'team1': 'BSC Young Boys',
  'team2': 'FC Zürich',
  'score': {'ft': [2, 1]}},
 {'round': 'Spieltag 1',
  'date': '2020-09-20',
  'team1': 'FC Basel',
  'team2': 'FC Vaduz',
  'score': {'ft': [2, 2]}}]

In [6]:
mh

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Switzerland,Super League,2012/2013,13/07/2012,18:45,Servette,Basel,0,1,A,6.22,4.17,1.60,6.22,4.20,1.68,5.32,3.85,1.59
1,Switzerland,Super League,2012/2013,14/07/2012,18:45,Thun,Lausanne,0,0,D,1.75,3.80,5.23,1.77,3.81,5.38,1.70,3.51,4.85
2,Switzerland,Super League,2012/2013,15/07/2012,12:45,Grasshoppers,Sion,0,2,A,3.27,3.36,2.39,3.35,3.41,2.41,3.07,3.24,2.25
3,Switzerland,Super League,2012/2013,15/07/2012,12:45,Luzern,Zurich,1,1,D,2.04,3.65,3.89,2.20,3.80,3.89,2.10,3.33,3.31
4,Switzerland,Super League,2012/2013,15/07/2012,15:00,St. Gallen,Young Boys,1,1,D,3.62,3.67,2.12,3.69,3.67,2.15,3.32,3.43,2.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802,Switzerland,Super League,2021/2022,22/05/2022,15:30,Sion,Servette,3,3,D,1.80,3.75,4.67,1.91,4.00,4.90,1.75,3.76,4.41
1803,Switzerland,Super League,2021/2022,22/05/2022,15:30,St. Gallen,Lausanne,4,0,H,1.41,5.34,7.12,1.50,5.90,7.50,1.38,5.35,6.47
1804,Switzerland,Super League,2021/2022,22/05/2022,15:30,Young Boys,Grasshoppers,3,0,H,1.31,6.66,7.75,1.35,7.00,9.50,1.28,6.17,8.13
1805,Switzerland,Super League,2021/2022,22/05/2022,15:30,Zurich,Luzern,2,3,A,2.32,4.10,2.80,2.34,4.20,2.91,2.25,3.97,2.71


In [7]:
mh_dict = mh.to_dict('records')

In [8]:
mh_dict[0]

{'Country': 'Switzerland',
 'League': 'Super League',
 'Season': '2012/2013',
 'Date': '13/07/2012',
 'Time': '18:45',
 'Home': 'Servette',
 'Away': 'Basel',
 'HG': 0,
 'AG': 1,
 'Res': 'A',
 'PH': 6.22,
 'PD': 4.17,
 'PA': 1.6,
 'MaxH': 6.22,
 'MaxD': 4.2,
 'MaxA': 1.68,
 'AvgH': 5.32,
 'AvgD': 3.85,
 'AvgA': 1.59}

In [9]:
adapter = mezzala.KeyAdapter(
    home_team='Home',
    away_team='Away',
    home_goals='HG',
    away_goals='AG'
)

In [10]:
model = mezzala.DixonColes(adapter,
blocks=[
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
    mezzala.blocks.BaseRate(),
])
model.fit(mh_dict)

DixonColes(adapter=KeyAdapter(home_goals='HG', away_goals='AG', home_team='Home', away_team='Away'), blocks=[TeamStrength(), HomeAdvantage(), BaseRate()]), weight=UniformWeight()

In [11]:
match_to_predict = {
    'Home': 'Basel',
    'Away': 'Wintherthur'
}

scorelines = model.predict_one(match_to_predict)

In [12]:
scorelines

[ScorelinePrediction(home_goals=0, away_goals=0, probability=0.04636930555473364),
 ScorelinePrediction(home_goals=0, away_goals=1, probability=0.03248236552122059),
 ScorelinePrediction(home_goals=0, away_goals=2, probability=0.016891972336856968),
 ScorelinePrediction(home_goals=0, away_goals=3, probability=0.005084197752980964),
 ScorelinePrediction(home_goals=0, away_goals=4, probability=0.0011476930998319242),
 ScorelinePrediction(home_goals=0, away_goals=5, probability=0.00020726171803675614),
 ScorelinePrediction(home_goals=0, away_goals=6, probability=3.119113446634735e-05),
 ScorelinePrediction(home_goals=0, away_goals=7, probability=4.023430080217539e-06),
 ScorelinePrediction(home_goals=0, away_goals=8, probability=4.541191319726359e-07),
 ScorelinePrediction(home_goals=0, away_goals=9, probability=4.556072403937591e-08),
 ScorelinePrediction(home_goals=0, away_goals=10, probability=4.113902026936687e-09),
 ScorelinePrediction(home_goals=0, away_goals=11, probability=3.37694

In [13]:
mh['Home'].unique

<bound method Series.unique of 0           Servette
1               Thun
2       Grasshoppers
3             Luzern
4         St. Gallen
            ...     
1802            Sion
1803      St. Gallen
1804      Young Boys
1805          Zurich
1806          Luzern
Name: Home, Length: 1807, dtype: object>

In [14]:
mezzala.scorelines_to_outcomes(scorelines)

{Outcomes('Home win'): OutcomePrediction(outcome=Outcomes('Home win'), probability=0.6789564408083062),
 Outcomes('Draw'): OutcomePrediction(outcome=Outcomes('Draw'), probability=0.1920213335733455),
 Outcomes('Away win'): OutcomePrediction(outcome=Outcomes('Away win'), probability=0.12902222561834814)}

In [15]:
matches_to_predict = [{
    'Home': 'Basel',
    'Away': 'Zurich'
},{
    'Home': 'Zurich',
    'Away': 'Basel'
},]
scorelines = model.predict(matches_to_predict)

In [16]:
scorelines

[[ScorelinePrediction(home_goals=0, away_goals=0, probability=0.04331448748398541),
  ScorelinePrediction(home_goals=0, away_goals=1, probability=0.03411600729728778),
  ScorelinePrediction(home_goals=0, away_goals=2, probability=0.02007900994952344),
  ScorelinePrediction(home_goals=0, away_goals=3, probability=0.006858610852408655),
  ScorelinePrediction(home_goals=0, away_goals=4, probability=0.001757079019696418),
  ScorelinePrediction(home_goals=0, away_goals=5, probability=0.0003601110193179256),
  ScorelinePrediction(home_goals=0, away_goals=6, probability=6.150356395496694e-05),
  ScorelinePrediction(home_goals=0, away_goals=7, probability=9.003624302132567e-06),
  ScorelinePrediction(home_goals=0, away_goals=8, probability=1.1533005518858207e-06),
  ScorelinePrediction(home_goals=0, away_goals=9, probability=1.3131523418631367e-07),
  ScorelinePrediction(home_goals=0, away_goals=10, probability=1.345644171511868e-08),
  ScorelinePrediction(home_goals=0, away_goals=11, probabil

In [17]:
final_score = scorelines[0][0]
for scores in scorelines[0]:
    if scores.probability > final_score.probability:
        final_score = scores

In [18]:
final_score.probability

0.0982345213997128

In [19]:
scorelines[0][0]

ScorelinePrediction(home_goals=0, away_goals=0, probability=0.04331448748398541)

In [20]:
df_schedule = fms.get_league_schedule(69)

getting fixtures for Super League


In [21]:
df_schedule

['id',
 'pageUrl',
 'displayTournament',
 'lnameArr',
 'color',
 'notStarted',
 'opponent.id',
 'opponent.name',
 'home.id',
 'home.name',
 'away.id',
 'away.name',
 'status.started',
 'status.cancelled',
 'status.finished',
 'status.startTimeStr',
 'status.startDateStr',
 'status.startDateStrShort']

In [22]:
league_url = "https://www.fotmob.com/api/leagues?id={}"
league_id = 69
response = requests.get(league_url.format(league_id)).json()
df_fixtures = pd.json_normalize(response, record_path=["matches"])

In [23]:
df_fixtures_short = df_fixtures[['id','home.name','away.name', 'home.id', 'away.id','status.startDateStr']]

In [24]:
df_fixtures_short['home.name'].unique()

array(['Young Boys', 'Winterthur', 'Lugano', 'Servette', 'FC Zürich',
       'St. Gallen', 'Basel', 'Grasshopper', 'Sion', 'Luzern'],
      dtype=object)

In [25]:
mh["Home"].unique()

array(['Servette', 'Thun', 'Grasshoppers', 'Luzern', 'St. Gallen',
       'Basel', 'Lausanne', 'Sion', 'Young Boys', 'Zurich', 'Aarau',
       'Vaduz', 'Lugano', 'Xamax'], dtype=object)

In [26]:
dict_teams = {
    'Young Boys':'Young Boys', 
    'Thun':'Winterthur', 
    'Lugano':'Lugano', 
    'Servette':'Servette', 
    'FC Zürich':'Zurich',
    'St. Gallen':'St. Gallen', 
    'Basel':'Basel', 
    'Grasshoppers':'Grasshopper', 
    'Sion':'Sion', 
    'Luzern':'Luzern',
    'Lausanne':'Winterthur',
    'Aarau':'Winterthur',
    'Vaduz':'Winterthur',
    'Xamax':'Winterthur'
}

In [27]:
mh['Home'] = mh['Home'].map(dict_teams)
mh['Away'] = mh['Away'].map(dict_teams)

In [28]:
mh = mh[mh['Home'] != mh['Away']]

In [30]:
df_fixtures_short

,id,home.name,away.name,home.id,away.id,status.startDateStr
0,3905393,Young Boys,FC Zürich,10192,10243,Tomorrow
1,3905401,Winterthur,Basel,7894,9931,Tomorrow
2,3905400,Lugano,Sion,7896,10179,"Sun, 17 Jul"
3,3905399,Servette,St. Gallen,9777,10190,"Sun, 17 Jul"
4,3905397,FC Zürich,Luzern,10243,10199,"Sat, 23 Jul"
...,...,...,...,...,...,...
175,3905541,Basel,Grasshopper,9931,9956,29 May 2023
176,3905573,FC Zürich,Lugano,10243,7896,29 May 2023
177,3905540,Luzern,Servette,10199,9777,29 May 2023
178,3905528,St. Gallen,Sion,10190,10179,29 May 2023


In [31]:
mh_dict = mh.to_dict('records')

In [32]:
model = mezzala.DixonColes(adapter,
blocks=[
    mezzala.blocks.HomeAdvantage(),
    mezzala.blocks.TeamStrength(),
    mezzala.blocks.BaseRate(),
])
model.fit(mh_dict)

DixonColes(adapter=KeyAdapter(home_goals='HG', away_goals='AG', home_team='Home', away_team='Away'), blocks=[TeamStrength(), HomeAdvantage(), BaseRate()]), weight=UniformWeight()

In [59]:
df_predictions = pd.DataFrame()
df_score_pred = pd.DataFrame()

for index, row in df_fixtures_short.iterrows():
    _match_to_predict = {
        'Home': row['home.name'],
        'Away': row['away.name']
    }

    _scorelines = model.predict_one(_match_to_predict)

    _pred = mezzala.scorelines_to_outcomes(_scorelines)

    _df_scores = pd.DataFrame()

    for scoreline in _scorelines:
        _dict_scorelines = {
            'home': [row['home.name']],
            'away': [row['away.name']], 
            'game_id': [row['id']],
            'date': [row['status.startDateStr']],
            'home_goals': [scoreline.home_goals],
            'away_goals': [scoreline.away_goals],
            'prob':[scoreline.probability]
        }
        _df_scores = pd.concat([_df_scores,pd.DataFrame(_dict_scorelines)], ignore_index=True, axis=0)
    _df_scores['home_score_pred'] = _df_scores['home_goals'] * _df_scores['prob']
    _df_scores['away_score_pred'] = _df_scores['away_goals'] * _df_scores['prob']
    df_score_pred = pd.concat([df_score_pred, _df_scores.groupby(['home','away','game_id','date']).sum().reset_index()], ignore_index=True, axis=0)

    



    
    _dict_pred = {
        'home': [row['home.name']],
        'away': [row['away.name']], 
        'game_id': [row['id']],
        'date': [row['status.startDateStr']],
        'prob_away': [_pred.popitem()[1].probability],
        'prob_draw': [_pred.popitem()[1].probability],
        'prob_home': [_pred.popitem()[1].probability],}
    _pred_series = pd.DataFrame(_dict_pred)

    df_predictions = pd.concat([df_predictions, _pred_series], ignore_index=True, axis=0)

df_score_pred['home_score_pred_int'] = df_score_pred['home_score_pred'].round(0).astype(int)
df_score_pred['away_score_pred_int'] = df_score_pred['away_score_pred'].round(0).astype(int)

In [60]:
df_predictions.head()

,home,away,game_id,date,prob_away,prob_draw,prob_home
0,Young Boys,FC Zürich,3905393,Tomorrow,0.187113,0.212224,0.600663
1,Winterthur,Basel,3905401,Tomorrow,0.593337,0.222316,0.184346
2,Lugano,Sion,3905400,"Sun, 17 Jul",0.307450,0.274232,0.418318
3,Servette,St. Gallen,3905399,"Sun, 17 Jul",0.345344,0.259114,0.395542
4,FC Zürich,Luzern,3905397,"Sat, 23 Jul",0.276392,0.249768,0.473841


In [61]:
df_predictions['points_home'] = 3 * df_predictions['prob_home'] + 1 * df_predictions['prob_draw']
df_predictions['points_away'] = 3 * df_predictions['prob_away'] + 1 * df_predictions['prob_draw']

In [62]:
df_points = pd.concat([df_predictions[['home','points_home']].groupby('home').sum(),df_predictions[['away','points_away']].groupby('away').sum()], axis=1)
df_points['points_total'] = df_points['points_home'] + df_points['points_away']

In [63]:
df_points.sort_values(by='points_total', ascending=False)

,points_home,points_away,points_total
Basel,38.774671,31.686191,70.460862
Young Boys,37.449952,30.073542,67.523494
FC Zürich,28.993763,21.684207,50.677969
Luzern,27.325882,20.015441,47.341323
Grasshopper,27.145804,19.961287,47.107090
St. Gallen,26.808713,19.475898,46.284610
Sion,25.741429,18.730548,44.471977
Lugano,25.105737,18.189134,43.294871
Servette,24.084107,17.157773,41.241880
Winterthur,22.268353,15.527351,37.795704


In [77]:
df_score_pred.head(10)

,home,away,game_id,date,home_goals,away_goals,prob,home_score_pred,away_score_pred,home_score_pred_int,away_score_pred_int
0,Young Boys,FC Zürich,3905393,Tomorrow,8450,8450,1.0,2.142445,1.127440,2,1
1,Winterthur,Basel,3905401,Tomorrow,8450,8450,1.0,1.037166,2.007723,1,2
2,Lugano,Sion,3905400,"Sun, 17 Jul",8450,8450,1.0,1.422664,1.185173,1,1
3,Servette,St. Gallen,3905399,"Sun, 17 Jul",8450,8450,1.0,1.508961,1.396395,2,1
4,FC Zürich,Luzern,3905397,"Sat, 23 Jul",8450,8450,1.0,1.704918,1.255118,2,1
5,St. Gallen,Winterthur,3905396,"Sat, 23 Jul",8450,8450,1.0,1.819204,1.157676,2,1
6,Basel,Servette,3905395,"Sun, 24 Jul",8450,8450,1.0,2.372555,0.878120,2,1
7,Grasshopper,Lugano,3905394,"Sun, 24 Jul",8450,8450,1.0,1.566153,1.136392,2,1
8,Sion,Young Boys,3905402,"Sun, 24 Jul",8450,8450,1.0,1.235642,1.803016,1,2
9,St. Gallen,FC Zürich,3905403,"Sat, 30 Jul",8450,8450,1.0,1.560600,1.417000,2,1
